In [ ]:
# Required Libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

# Install and Import TOPSIS Package
!pip install topsis-manya-102203284
from topsis_manya.topsis import topsis  

# Function to Evaluate Models
def evaluate_model(model_name, conversations):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    embeddings = []
    for text in conversations:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.logits.mean(dim=1).numpy())

    return cosine_similarity(np.vstack(embeddings))

# Sample Conversations
conversations = [
    "What's your favorite book?",
    "Explain the theory of relativity in simple words.",
    "Tell me a fun fact!",
    "Give me some motivation to study.",
    "Translate 'Good morning' to French."
]

# Selected Lightweight Models
models = ["distilgpt2", "facebook/blenderbot-90M", "microsoft/DialoGPT-small"]

# Evaluate Models and Store Results
model_scores = []
for model in models:
    print(f"Processing: {model}")
    similarity = evaluate_model(model, conversations)
    
    model_scores.append({
        "Model": model,
        "Mean Similarity": similarity.mean(),
        "Inference Speed": np.random.uniform(0.9, 1.0),
        "Model Size": np.random.uniform(0.7, 1.0)
    })

# Save Evaluations
df = pd.DataFrame(model_scores)
df.to_csv("input.csv", index=False)

# TOPSIS Parameters
weights = [1, 1, 1]
impacts = ['+', '+', '-']

# Apply TOPSIS
topsis("input.csv", weights, impacts, "output.csv")

# Read and Sort TOPSIS Results
ranked_df = pd.read_csv("output.csv")

if "Topsis Score" in ranked_df.columns:
    score_col = "Topsis Score"
else:
    score_col = ranked_df.columns[-1]

ranked_df = ranked_df.sort_values(by=score_col, ascending=False)
ranked_df["Rank"] = range(1, len(ranked_df) + 1)

# Display and Save Results
display(ranked_df)
ranked_df.to_csv("corrected_topsis_results.csv", index=False)

# Visualization
plt.figure(figsize=(8, 4))
sns.barplot(x=ranked_df["Model"], y=ranked_df[score_col], palette="coolwarm")

plt.xlabel("Pre-trained Models")
plt.ylabel("Topsis Score")
plt.title("Model Ranking using TOPSIS")
plt.xticks(rotation=20)
plt.savefig("corrected_topsis_results_graph.png", format="png")
plt.show()
